<a href="https://colab.research.google.com/github/killskaii/M1L1/blob/main/%D0%9B%D0%B0%D0%B12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import itertools
from collections import defaultdict

#Задача 1
def truth_table_by_func(f, n=3):
    "Генерация таблицы истинности для функции n переменных"
    tt = {}
    for vals in itertools.product([0,1], repeat=n):
        tt[vals] = f(*vals)
    return tt

def fdnf(tt):
    "Построение СДНФ по таблице истинности"
    terms = []
    for vars, val in tt.items():
        if val == 1:
            term = []
            for i, v in enumerate(vars):
                if v == 1:
                    term.append(f"x{i+1}")
                else:
                    term.append(f"¬x{i+1}")
            terms.append(" ∧ ".join(term))
    return " ∨ ".join(terms) if terms else "0"

def reduce_dnf(tt):
    "Сокращение ДНФ методом Квайна"
    minterms = [vars for vars, val in tt.items() if val == 1]
    prime_implicants = set()
    changed = True

    while changed:
        changed = False
        new_implicants = set()
        used = set()

        for i in range(len(minterms)):
            for j in range(i+1, len(minterms)):
                diff = [k for k in range(len(minterms[i]))
                       if minterms[i][k] != minterms[j][k]]

                if len(diff) == 1:
                    k = diff[0]
                    new_term = list(minterms[i])
                    new_term[k] = -1
                    new_implicants.add(tuple(new_term))
                    used.add(i)
                    used.add(j)
                    changed = True

        for i in range(len(minterms)):
            if i not in used:
                prime_implicants.add(tuple(minterms[i]))

        minterms = list(new_implicants)
        prime_implicants.update(new_implicants)

    def term_to_str(term):
        parts = []
        for i, v in enumerate(term):
            if v == 1:
                parts.append(f"x{i+1}")
            elif v == 0:
                parts.append(f"¬x{i+1}")
        return " ∧ ".join(parts) if parts else "1"

    return " ∨ ".join(term_to_str(t) for t in prime_implicants)

#Задача 2
def zhegalkin_polynomial(tt):
    "Построение многочлена Жегалкина методом треугольника"
    n = len(next(iter(tt.keys())))
    values = [tt[tuple(vars)] for vars in itertools.product([0,1], repeat=n)]

    triangle = [values.copy()]
    for i in range(1, 2**n):
        new_row = []
        for j in range(len(triangle[-1]) - 1):
            new_row.append(triangle[-1][j] ^ triangle[-1][j+1])
        triangle.append(new_row)

    polynom = []
    for i, vars in enumerate(itertools.product([0,1], repeat=n)):
        if triangle[i][0] == 1:
            term = []
            for j, v in enumerate(vars):
                if v == 1:
                    term.append(f"x{j+1}")
            if term:
                polynom.append(" ∧ ".join(term))
            else:
                polynom.append("1")

    return " ⊕ ".join(polynom) if polynom else "0"

#Задача 3
def check_post_classes(functions):
    "Проверка полноты системы функций по критерию Поста"
    classes = {
        'T0': lambda f: f(*([0]*f.__code__.co_argcount)) == 0,
        'T1': lambda f: f(*([1]*f.__code__.co_argcount)) == 1,
        'S': lambda f: all(not f(*[1-v for v in vars]) == f(*vars)
                          for vars in itertools.product([0,1], repeat=f.__code__.co_argcount)),
        'L': lambda f: is_linear(f),
        'M': lambda f: is_monotone(f)
    }

    results = defaultdict(list)
    for name, test in classes.items():
        for func in functions:
            try:
                if not test(func):
                    results[name].append(func.__name__)
            except TypeError:
                # Пропускаем функции с неподходящим числом аргументов
                continue

    is_complete = all(len(funcs) > 0 for funcs in results.values())
    return is_complete, results

def is_linear(f):
    "Проверка линейности функции"
    try:
        n = f.__code__.co_argcount
        tt = truth_table_by_func(f, n)
        poly = zhegalkin_polynomial(tt)
        return ' ∧ ' not in poly
    except TypeError:
        return False

def is_monotone(f):
    "Проверка монотонности функции"
    try:
        n = f.__code__.co_argcount
        for vars1 in itertools.product([0,1], repeat=n):
            for vars2 in itertools.product([0,1], repeat=n):
                if all(v1 <= v2 for v1, v2 in zip(vars1, vars2)):
                    if f(*vars1) > f(*vars2):
                        return False
        return True
    except TypeError:
        return False

# Пример использования:
if __name__ == "__main__":
    # Пример функции для анализа (мажоритарная функция)
    def maj(x, y, z):
        return (x & y) | (x & z) | (y & z)

    # Задача 1
    print("=== Задача 1 ===")
    tt = truth_table_by_func(maj)
    print("Таблица истинности:")
    for vars, val in sorted(tt.items()):
        print(f"{vars}: {val}")

    print("\nСДНФ:")
    sdnf = fdnf(tt)
    print(sdnf)

    print("\nСокращенная ДНФ:")
    reduced = reduce_dnf(tt)
    print(reduced)

    # Задача 2
    print("\n=== Задача 2 ===")
    print("Многочлен Жегалкина:")
    poly = zhegalkin_polynomial(tt)
    print(poly)

    # Задача 3
    print("\n=== Задача 3 ===")
    # Пример системы функций (теперь все функции от 2 переменных)
    def f1(x, y): return not x
    def f2(x, y): return x and y
    def f3(x, y): return x or y

    is_complete, results = check_post_classes([f1, f2, f3])
    print("Результаты проверки по классам Поста:")
    for cls, funcs in results.items():
        print(f"{cls}: {funcs or 'нет функций вне класса'}")
    print(f"\nСистема {'полна' if is_complete else 'не полна'}")

=== Задача 1 ===
Таблица истинности:
(0, 0, 0): 0
(0, 0, 1): 0
(0, 1, 0): 0
(0, 1, 1): 1
(1, 0, 0): 0
(1, 0, 1): 1
(1, 1, 0): 1
(1, 1, 1): 1

СДНФ:
¬x1 ∧ x2 ∧ x3 ∨ x1 ∧ ¬x2 ∧ x3 ∨ x1 ∧ x2 ∧ ¬x3 ∨ x1 ∧ x2 ∧ x3

Сокращенная ДНФ:
x1 ∧ x2 ∨ x1 ∧ x3 ∨ x2 ∧ x3

=== Задача 2 ===
Многочлен Жегалкина:
x2 ∧ x3 ⊕ x1 ∧ x3 ⊕ x1 ∧ x2

=== Задача 3 ===
Результаты проверки по классам Поста:
T0: ['f1']
T1: ['f1']
S: ['f2', 'f3']
L: ['f2', 'f3']
M: ['f1']

Система полна
